# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
%config Completer.use_jedi = False

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
#loading csv from weather.py portion

city_data_weather_df = pd.read_csv('../output_data/cities.csv')
city_data_weather_df


,Unnamed: 0,City,Lat,Long,Temp. Max,Humidity,Cloudiness,Windspeed,description,Country
0,0,punta arenas,-53.1500,-70.9167,32.00,69,75,8.05,broken clouds,CL
1,1,kangaba,11.9333,-8.4167,91.90,41,80,3.22,broken clouds,ML
2,2,kodiak,57.7900,-152.4072,39.20,87,90,14.97,light rain,US
3,3,guilin,25.2819,110.2864,80.60,78,75,8.95,broken clouds,CN
4,4,zhezkazgan,47.8043,67.7144,73.40,11,100,10.38,overcast clouds,KZ
...,...,...,...,...,...,...,...,...,...,...
555,555,asfi,32.2994,-9.2372,64.80,56,99,14.47,overcast clouds,MA
556,556,oga,46.4631,10.3470,51.01,38,0,5.75,clear sky,IT
557,557,nador,35.1740,-2.9287,62.60,82,75,10.36,broken clouds,MA
558,558,amazar,53.8580,120.8750,41.90,92,100,18.25,moderate rain,RU


In [23]:
# checking for missing values
city_data_weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   560 non-null    int64  
 1   City         560 non-null    object 
 2   Lat          560 non-null    float64
 3   Long         560 non-null    float64
 4   Temp. Max    560 non-null    float64
 5   Humidity     560 non-null    int64  
 6   Cloudiness   560 non-null    int64  
 7   Windspeed    560 non-null    float64
 8   description  560 non-null    object 
 9   Country      556 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 43.9+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key)

In [25]:
locations = city_data_weather_df[["Lat", "Long"]]
humidity = city_data_weather_df["Humidity"]


In [26]:
figure =gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
figure.add_layer(heat_layer)



figure

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
#new df filtered from previous df
city_data_weather_df.head()

,Unnamed: 0,City,Lat,Long,Temp. Max,Humidity,Cloudiness,Windspeed,description,Country
0,0,punta arenas,-53.1500,-70.9167,32.0,69,75,8.05,broken clouds,CL
1,1,kangaba,11.9333,-8.4167,91.9,41,80,3.22,broken clouds,ML
2,2,kodiak,57.7900,-152.4072,39.2,87,90,14.97,light rain,US
3,3,guilin,25.2819,110.2864,80.6,78,75,8.95,broken clouds,CN
4,4,zhezkazgan,47.8043,67.7144,73.4,11,100,10.38,overcast clouds,KZ


In [28]:
(city_data_weather_df['Temp. Max'] <80) & (city_data_weather_df['Temp. Max'] >=70)

0      False
1      False
2      False
3      False
4       True
       ...  
555    False
556    False
557    False
558    False
559    False
Name: Temp. Max, Length: 560, dtype: bool

In [29]:
(city_data_weather_df['Windspeed']<10)

0       True
1       True
2      False
3       True
4      False
       ...  
555    False
556     True
557    False
558    False
559    False
Name: Windspeed, Length: 560, dtype: bool

In [30]:
(city_data_weather_df['Cloudiness'] == 0)

0      False
1      False
2      False
3      False
4      False
       ...  
555    False
556     True
557    False
558    False
559    False
Name: Cloudiness, Length: 560, dtype: bool

In [31]:
#filtering using .loc 
ideal_vacay_spot = city_data_weather_df.loc[
    (city_data_weather_df['Temp. Max'] <80) & (city_data_weather_df['Temp. Max'] >=70) \
    & (city_data_weather_df['Windspeed']<10) \
    & (city_data_weather_df['Cloudiness'] == 0) 


]
ideal_vacay_spot

,Unnamed: 0,City,Lat,Long,Temp. Max,Humidity,Cloudiness,Windspeed,description,Country
41,41,cabedelo,-6.9811,-34.8339,75.20,88,0,5.75,clear sky,BR
68,68,east london,-33.0153,27.9116,75.20,53,0,9.22,clear sky,ZA
141,141,santa maria,-29.6842,-53.8069,71.60,68,0,2.30,clear sky,BR
165,165,port elizabeth,-33.9180,25.5701,73.40,49,0,6.91,clear sky,ZA
166,166,luderitz,-26.6481,15.1594,74.39,53,0,6.67,clear sky,NaN
226,226,tautira,-17.7333,-149.1500,77.00,78,0,5.75,clear sky,PF
347,347,teahupoo,-17.8333,-149.2667,77.00,78,0,5.75,clear sky,PF
350,350,ulundi,-28.3352,31.4162,70.86,52,0,5.21,clear sky,ZA
510,510,umtata,-31.5889,28.7844,73.40,38,0,2.30,clear sky,ZA


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
ideal_vacay_spot.head(1)

,Unnamed: 0,City,Lat,Long,Temp. Max,Humidity,Cloudiness,Windspeed,description,Country
41,41,cabedelo,-6.9811,-34.8339,75.2,88,0,5.75,clear sky,BR


In [33]:
ideal_vacay_spot_dh = ideal_vacay_spot.copy()

In [34]:
ideal_vacay_spot_dh['Hotel Name'] = ""
ideal_vacay_spot_dh

,Unnamed: 0,City,Lat,Long,Temp. Max,Humidity,Cloudiness,Windspeed,description,Country,Hotel Name
41,41,cabedelo,-6.9811,-34.8339,75.20,88,0,5.75,clear sky,BR,
68,68,east london,-33.0153,27.9116,75.20,53,0,9.22,clear sky,ZA,
141,141,santa maria,-29.6842,-53.8069,71.60,68,0,2.30,clear sky,BR,
165,165,port elizabeth,-33.9180,25.5701,73.40,49,0,6.91,clear sky,ZA,
166,166,luderitz,-26.6481,15.1594,74.39,53,0,6.67,clear sky,NaN,
226,226,tautira,-17.7333,-149.1500,77.00,78,0,5.75,clear sky,PF,
347,347,teahupoo,-17.8333,-149.2667,77.00,78,0,5.75,clear sky,PF,
350,350,ulundi,-28.3352,31.4162,70.86,52,0,5.21,clear sky,ZA,
510,510,umtata,-31.5889,28.7844,73.40,38,0,2.30,clear sky,ZA,


In [35]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

In [36]:
# Use the lat/lng we recovered to identify hotels
for index, row in ideal_vacay_spot_dh.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    
    try:
        ideal_vacay_spot_dh.loc[index, "Hotel Name"] = name_address['results'][0]['name']
    
    except ():
        print("missing results")

ideal_vacay_spot_dh        


,Unnamed: 0,City,Lat,Long,Temp. Max,Humidity,Cloudiness,Windspeed,description,Country,Hotel Name
41,41,cabedelo,-6.9811,-34.8339,75.20,88,0,5.75,clear sky,BR,Hotel Pigalle
68,68,east london,-33.0153,27.9116,75.20,53,0,9.22,clear sky,ZA,Tu Casa
141,141,santa maria,-29.6842,-53.8069,71.60,68,0,2.30,clear sky,BR,Hotel Morotin
165,165,port elizabeth,-33.9180,25.5701,73.40,49,0,6.91,clear sky,ZA,39 On Nile Guest House
166,166,luderitz,-26.6481,15.1594,74.39,53,0,6.67,clear sky,NaN,LÜDERITZ NEST HOTEL
226,226,tautira,-17.7333,-149.1500,77.00,78,0,5.75,clear sky,PF,Pension Oaoa
347,347,teahupoo,-17.8333,-149.2667,77.00,78,0,5.75,clear sky,PF,Vanira Lodge
350,350,ulundi,-28.3352,31.4162,70.86,52,0,5.21,clear sky,ZA,Ekhethelo Lodge Ulundi
510,510,umtata,-31.5889,28.7844,73.40,38,0,2.30,clear sky,ZA,LLZZ Boutique Lodge


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_vacay_spot_dh.iterrows()]
locations = ideal_vacay_spot_dh[["Lat", "Long"]]

In [39]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

figure.add_layer(marker_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))